<a href="https://colab.research.google.com/github/nadia-maarfavi/Video.Analysis/blob/main/LowLevelVideoFeatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
import numpy as np
import cv2
import time
import os
from csv import writer
import sys

In [10]:
def image_colorfulness(image):
    # split the image into its respective RGB components
    (B, G, R) = cv2.split(image.astype("float"))
    # compute rg = R - G
    rg = np.absolute(R - G)
    # compute yb = 0.5 * (R + G) - B
    yb = np.absolute(0.5 * (R + G) - B)
    # compute the mean and standard deviation of both `rg` and `yb`
    (rbMean, rbStd) = (np.mean(rg), np.std(rg))
    (ybMean, ybStd) = (np.mean(yb), np.std(yb))
    # combine the mean and standard deviations
    stdRoot = np.sqrt((rbStd ** 2) + (ybStd ** 2))
    meanRoot = np.sqrt((rbMean ** 2) + (ybMean ** 2))
    # derive the "colorfulness" metric and return it
    color = stdRoot + (0.3 * meanRoot)
    return color

In [11]:
def image_brightness(image):
    # calculate mean brightness in HSV space
    bright_pixel = round(image[..., 2].mean(), 4)
    return bright_pixel

In [12]:
def image_saturation(image):
    saturation = round(image[:, :, 1].mean(), 4)
    return saturation

In [13]:
def image_contrast(image):
    img_grey = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    contrast = img_grey.std()
    return contrast

In [20]:
def lowlevel(f1):

    Brightness = []
    Saturation = []
    Contrast = []
    ColorFulness = []
    AllFeatures = []

    cap = cv2.VideoCapture(f1)

    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count / fps

    ret, frame = cap.read()


    while (True):

      gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
      gray = cv2.GaussianBlur(gray, (21, 21), 0)

      # calculate colorfulness in HSV space
      ColorFulness.append(image_colorfulness(frame))

      # convert to hsv
      img_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

      # calculate mean brightness in HSV space
      Brightness.append(image_brightness(img_hsv))

      # calculate saturation in HSV space
      Saturation.append(image_saturation(img_hsv))

      # calculate contrast
      Contrast.append(image_contrast(frame))

      ret, frame = cap.read()
      if ret == False:
        print("Video Finished !!!")
        break

    cap.release()

    #print(len(Brightness), len(Saturation), len(Contrast), len(ColorFulness), len(MotionScores))
    #print((Brightness), (Saturation), (Contrast), (ColorFulness), (MotionScores))

    AllFeatures.extend([duration, np.mean(Brightness),np.var(Brightness),np.mean(Saturation),np.var(Saturation),np.mean(Contrast),np.var(Contrast),
                       np.mean(ColorFulness),np.var(ColorFulness)])
    return AllFeatures

In [21]:
videopath = "/content/drive/MyDrive/Video/Sample1.mp4"

if((os.path.isfile(videopath))==False):
  print("Video not found")

else:
  OneVideoFeature = lowlevel(videopath)

print(OneVideoFeature)

Video Finished !!!
[9.008999999999999, 77.61578425925926, 103.44392944280779, 67.5974712962963, 286.56929483139834, 60.510023897902116, 180.9932829107019, 19.90673739233737, 33.41842554847253]
